## MSTICPy and Notebooks in InfoSec

---

<h1 style="border: solid; padding:5pt; color:black; background-color:#909090">Session 4 - Enrichment and Context</h1>

---

## What this session covers:

- Pivot functions
  - Basics
  - Data queries
  - Chained pivot functions
- Beyond pivots - Threat Intelligence providers
- Azure APIs - Sentinel


## Prerequisites
- Python >= 3.8 Environment
- Jupyter installed
- MSTICPy installed
- Run az login

## Recommended
- VS Code


---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Notebook Setup</a>

---

In [ ]:
%env MSTICPYCONFIG=./msticpyconfig.yaml
import msticpy as mp
mp.init_notebook()

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">What is enrichment?</a>

---

### Answering questions about an entity to give you insight into intent and activity


In [ ]:
from IPython.display import Image, Markdown
from time import sleep

display(Markdown("## Answering questions about an entity to give you insight into intent and activity"))
sleep(1)
display(Image("./media/enrichment_1.png", width="70%"))

In [ ]:
display(Markdown("## ...and for a whole range of entities"))

for entity in dir(mp.entities):
    cls = getattr(mp.entities, entity)
    if isinstance(cls, type) and issubclass(cls, mp.entities.Entity):
        print(entity, end="  ")
        sleep(0.1)

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">MSTICPy Pivots</a>

---

## **Pivots** or **Pivot Functions** are contextual functions attached to **Entities**
### [Reference: Pivot Documentation](https://msticpy.readthedocs.io/en/latest/data_analysis/PivotFunctions.html)

In [ ]:
IpAddress.pivots()

In [ ]:
mp.Pivot().browse()

## Example utility functions

- Entities are auto-imported by `mp.init_notebook()`
- Support auto/tabbed completion

In [ ]:
IpAddress

In [ ]:
IpAddress.whois("54.69.246.204")

In [ ]:
Dns.util.dns_components("www.microsoft.com")

In [ ]:
Url.util.url_components("https://ms.web.azuresynapse.net/en/authoring/orchestrate/pipeline/Notebook%20test%20simple?workspace=%2Fsubscriptions%2F40dcc8bf-0478-4f3b-b275-ed0a94f2c013%2FresourceGroups%2FASIHuntOMSWorkspaceRG%2Fproviders%2FMicrosoft.Synapse%2Fworkspaces%2Fianhelle-synapse1&livyId=18&sparkPoolName=ianhellespark1&snapshotId=d941f342-272e-4898-9eee-19aa90f05f44")

In [ ]:
Dns.util.dns_resolve("www.microsoft.com")

## Threat Intel providers

In [ ]:
IpAddress.tilookup_ip("54.69.246.204") #, providers=["VirusTotal"])

## Support lists as inputs

### Actually support iterables: list, tuple, set, generator, pandas series.

In [ ]:
IpAddress.geoloc(["54.69.246.204", "104.73.1.162"])

## DataFrames as inputs

In [ ]:
qry_local = mp.QueryProvider("LocalData")
ip_flow_df = qry_local.Network.list_azure_network_flows_by_host()
ip_flow_df.head(2)

In [ ]:
IpAddress.whois(ip_flow_df.head(5), column="AllExtIPs")

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 1 - Find WhoIs and GeoLocation info for Domain/Hosts</a>

1. Use pivot function to resolve domains (`Dns`) to IP Addresses
2. Find the Geo Location and WhoIs info for these addresses.

<details>
<summary>Hints...</summary>
<ul>
<li>Use the Dns class to resolve the domains to IP Addresses.</li>
<li>The IP address is returned in the 'rrest' column</li>
<li>Use the IpAddress "whois" and "geoip" functions to find the details</li>
</ul>
</details>

In [ ]:
domains = [
    "www.microsoft.com",
    "python.org",
    "kexp.org"
]

dns_df = # resolve domains
dns_df

In [ ]:
# show whois results

# show geolocation resolution

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Pivot functions can be queries.</a>

---

## Loading data providers adds new pivot functions dynamically

### Previous IP Address pivots
```python
['geoloc',
 'ip_type',
 'ti.lookup_ip',
 'tilookup_ip',
 'util.geoloc',
 'util.geoloc_ips',
 'util.ip_rev_resolve',
 'util.ip_type',
 'util.whois',
 'util.whois_asn',
 'whois',
 'whois_asn']
```

### Create a Data provider

In [ ]:
qry_prov = mp.QueryProvider("MSSentinel")
qry_prov.connect(workspace="Default")

In [ ]:
IpAddress.pivots()

## Adding queries is a very easy way of adding custom pivot functions

### You can also add arbitrary functions (see appendix)

If you create a query with one of these parameters it will be automatically added
to the mapped entity as a pivot function.

| Query Parameter  | Entity                               |
|------------------|--------------------------------------|
| account_name     | Account                              |
| user             | Account                              |
| host_name        | Host                                 |
| process_name     | Process                              |
| source_ip_list   | IpAddress                            |
| ip_address_list  | IpAddress                            |
| ip_address       | IpAddress                            |
| logon_session_id | Process, , HostLogonSession, Account |
| process_id       | Process                              |
| commandline      | Process                              |
| user             | Account                              |
| url              | Url                                  |
| file_hash        | File                                 |
| domain           | Dns                                  |
| resource_id      | AzureResource                        |


---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Chaining Pivot functions</a>

---

### Because Pivot functions use DataFrames as output and input you can use them in a pipeline

### Use the Pandas `mp_pivot.run()` function to add steps to the pipeline


```python
# Take 3 functions from previous task
Dns.util.dns_resolve(domains)
IpAddress.whois(dns_df, column="rrset")
IpAddress.geoloc(dns_df, column="rrset")
```

In [ ]:
# Use .mp_pivot.run to pipe output of first to input of second function
(
    Dns.util.dns_resolve(domains)
    .mp_pivot.run(IpAddress.whois, column="rrset")
)

In [ ]:
# Add this third function
# (note from the output, this function doesn't preserve input columns)
(
    Dns.util.dns_resolve(domains)
    .mp_pivot.run(IpAddress.whois, column="rrset")
    .mp_pivot.run(IpAddress.geoloc, column="rrset")
)

In [ ]:
# Add a join parameter
(
    Dns.util.dns_resolve(domains)
    .mp_pivot.display()  # add a display() function to display intermediate results
    .mp_pivot.run(IpAddress.whois, column="rrset")
    .mp_pivot.run(IpAddress.geoloc, column="rrset", join="left")
)

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Beyond Pivots</a>

---

## Looking up mixed type TI observables

### [Reference - Threat Intel Lookup](https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html)

In [ ]:
ioc_df = pd.read_csv("./data/cobalt_strike_c2_otx.csv")
ioc_df["Indicator type"].value_counts()

In [ ]:
ti_lookup = mp.TILookup()
ti_results = ti_lookup.lookup_iocs(data=ioc_df.sample(10), obs_col="Indicator")
ti_results.query("Severity == 'high'")

In [ ]:
TILookup.browse(ti_results)

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 4 (Optional if we have time) Find which IoCs have TI entries and resolve to a certain country</a>

Use a subset of the cobalt strike data and answer the following questions

1. How many of the IoCs have TI entries?
2. If you have configured multiple providers
   - Do the same observables appear in all providers?
   - Do they have the same severity?
3. What does the geographic distribution of the observables look like?
   - What is the most common origin country?

For the last question you will need to resolve the host/URL to an IP address.


In [ ]:
# your answer here

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Sentinel APIs</a>

---

### [Reference - MSTICPy Sentinel APIs](https://msticpy.readthedocs.io/en/latest/data_acquisition/Sentinel.html)

In [ ]:
sentinel = mp.MicrosoftSentinel()
sentinel.connect(workspace="ASIWorkspace")


In [ ]:
sentinel.list_incidents().head(2)

In [ ]:
sentinel.list_data_connectors().head(2)

In [ ]:
sentinel.list_hunting_queries().head(3)

In [ ]:
help(sentinel.create_incident)

In [ ]:
incident = sentinel.create_incident(
    title="Test incident (ianhelle)",
    severity="Informational",
    description="Incident caused by testing API.",
)

In [ ]:
sentinel.get_incident(incident)


In [ ]:
sentinel.get_incident_comments(incident)

In [ ]:
sentinel.post_comment(
    incident,
    comment="I have something to add..."
)

In [ ]:
sentinel.get_incident_comments(incident)

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 5 - Create and update an incident.</a>

Create an incident in Sentinel and update properties

1. Create an instance of the MicrosoftSentinel provider and connect to the "SentinelTest" workspace.
2. Create an incident
3. Retrieve the incident to confirm it is there
4. Add a comment and verify that it has been posted
5. Change the incident severity to Medium

Incident properties
https://docs.microsoft.com/rest/api/securityinsights/stable/incidents/create-or-update

<details>
<summary>Hints...</summary>
<ul>
<li>To create an incident, use <pre>sentinel.create_incident()</pre>
<ul><li>This returns the incident ID if successful.</li>
<li>Supply "title", "severity" and "description" parameters.</li>
</ul><br></li>
<li>To retrieve an incident (use either incident ID or incident title), use<pre>sentinel.get_incident(incident)</pre></li>
<li>To post a comment use<pre>sentinel.post_comment(incident, comment="My comment")</pre></li>
<li>To change the incident severity you need to use a dictionary
<pre>prop_dict = {"properties": {"severity": "Medium"}}
</pre>
</li>
<li>Pass this to <pre>sentinel.update_incident(incident, update_items=prop_dict)</pre>
</ul>
</details>


In [ ]:
sentinel = mp.MicrosoftSentinel()
sentinel.connect(workspace="SentinelTest")

# create the incident
incident = sentinel.create_incident(
    # params
)

In [ ]:
# update severity

# show the updated incident
sentinel.get_incident(incident)

In [ ]:
# post comment 

# and show new comments
sentinel.get_incident_comments(incident)

---
# End of Session


---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Appendix - Adding a custom pivot function</a>

---

### Here is the function that we want to add to some entities

Note it takes a string parameter for the entity value input and returns a string


In [ ]:
def defang_ioc(ioc: str, ioc_type: str = None) -> str:
    """
    Return de-fanged observable.

    Parameters
    ----------
    ioc : str
        The observable.
    ioc_type : str
        The type of IoC. If URL or Email it will do
        extra processing to neuter the URL protocol and email @ symbol

    Returns
    -------
    str
        The de-fanged observable.
    """
    de_fanged = ioc
    if ioc_type == "email":
        de_fanged = de_fanged.replace("@", "AT")
    elif ioc_type == "url":
        de_fanged = de_fanged.replace("http", "hXXp").replace("ftp", "fXp")
    return de_fanged.replace(".", "[.]")

### Call `Pivot.add_pivot_function` to add the function to a couple of entities

In [ ]:
from msticpy.init.pivot import PivotRegistration

mp.Pivot.add_pivot_function(
    func=defang_ioc,
    container="util",
    input_type="value",
    entity_map={
        "IpAddress": "Address",
        "Dns": "Domain",
    },
    func_input_value_arg="ioc",
    func_new_name="defang",
)

### Now we can defang IP addresses and DNS names

#### Note - even though the input and output of our original function was a string, it accepts lists and DataFrames as inputs.

In [ ]:
IpAddress.util.defang(["54.69.246.204", "104.73.1.162"])

### For URLs we want to also set the `ioc_type` parameter

We can add that as a registration parameter `func_static_params`.

In [ ]:
# Adding static parameters to supply ioc_type param
mp.Pivot.add_pivot_function(
    func=defang_ioc,
    container="util",
    input_type="value",
    entity_map={
        "Url": "Url",
    },
    func_input_value_arg="ioc",
    func_new_name="defang",
    func_static_params={"ioc_type": "url"}
)

In [ ]:
Url.util.defang("https//some.bad.stuff.org/deeppath?query=foo")